# VIK Data Cleaning

1. Remember to set Python kernel to 3 (not later).
2. Install additional packages `textblob`, `wordcloud`, and `gensim`.

### Import packages for scraping webpage contents and making sense of them

import requests
from bs4 import BeautifulSoup
import pickle

In [1]:
### Create variables and lists to serve as argument placeholders for scraping

#### `get` works in conjunction with `requests`.
#### `BeautifulSoup` must have a particular HTML element from a webpage to work on.  In this case, `<div class="post-content">`, and the *"p"* is from `<p style=...>`.

def url_to_transcript(url):
    '''Returns HTML contents of specified site.'''
    page = requests.get(url).text
    soup = BeautifulSoup(page, "lxml")
    text = [p.text for p in soup.find(class_="post-content").find_all('p')]
    print(url)
    return text

In [2]:
### Populate list of URLs

urls = ['http://scrapsfromtheloft.com/2017/05/06/louis-ck-oh-my-god-full-transcript/']

In [3]:
### Actually perform scrape of contents of scrapsfromtheloft.com

transcripts = [url_to_transcript(u) for u in urls]

NameError: name 'requests' is not defined